### Playground

This notebook shows how to communicate with configuration registers, status registers and AXI4-Stream ports inside the FPGA using the [pyhubio](https://github.com/pavel-demin/pyhubio) library.

Import the required libraries and instantiate `PyhubJTAG` and `PyhubIO`.

In [ ]:
from pyhubio import PyhubJTAG, PyhubIO
import numpy as np
import pylab as pl
import time

jtag = PyhubJTAG()
io = PyhubIO()

Program the FPGA.

In [ ]:
jtag.program("playground.bit")

Start I/O.

In [ ]:
io.start()
io.flush()

The port number (hub address) of the configuration registers is 0.

The first four bits of the 32-bit configuration register at address 0 are connected to the LEDs on the USB104 A7 board.

The following script turns all the LEDs on and off several times.

In [ ]:
leds = np.zeros(1, np.uint32)

for i in range(10):
    leds[0] ^= 15
    io.write(leds, port=0, addr=0)
    time.sleep(0.5)

The port number (hub address) of the status registers is 1.

The first two bits of the 32-bit status register at address 0 are connected to the buttons on the USB104 A7 board.

The following script reads the state of the buttons.

In [ ]:
buttons = np.zeros(1, np.uint32)

io.read(buttons, port=1, addr=0)

print("BTN0:", buttons[0] & 1, "BTN1:", buttons[0] >> 1)

The 32-bit configuration register at address 1 is connected to the 16-bit inputs of the DSP48 multiplier and the status register at address 1 is connected to the 32-bit output of the multiplier.

The following script writes two 16-bit integer values to be multiplied in the configuration register at address 1 and reads the result of the multiplication from the status register at address 1.

In [ ]:
input = np.array([6, 7], np.int16)
io.write(input, port=0, addr=1)

output = np.zeros(1, np.int32)
io.read(output, port=1, addr=1)

print(output[0])

The first AXI4-Stream interface has port number (hub address) 2. Its slave side is connected to a counter that increments after each read transaction.

The following script reads 10 numbers from the counter.

In [ ]:
output = np.zeros(10, np.int32)
io.read(output, port=2, addr=0)

print(output)

The second AXI4-Stream interface has port number (hub address) 3. Its slave side is connected to a direct digital synthesizer (DDS) that generates sine and cosine waveforms. The frequency is controlled by the 30-bit phase increment in the configuration register at address 2.

The following script sets the frequency to 1 MHz and reads 2048 samples from the DDS.

In [ ]:
freq = 1e6
io.write(np.uint32([freq / 100e6 * (1 << 30) + 0.5]), port=0, addr=2)

samples = np.zeros(2048, np.int16)
io.read(samples, port=3, addr=0)

cos = samples[0::2]
sin = samples[1::2]

pl.figure(figsize=[8, 4], dpi=150, constrained_layout=True)

pl.plot(cos)
pl.plot(sin)

pl.ylim(-35000, 35000)
pl.grid()

pl.show()

Stop I/O.

In [ ]:
io.stop()